In [1]:
from scipy.stats import norm
from sklearn.decomposition import PCA
from datetime import datetime, timedelta
import refinitiv.data as rd
import pandas as pd
import numpy as np
import random
import cufflinks as cf
import warnings
warnings.filterwarnings('ignore')


cf.go_offline()
rd.open_session()

<refinitiv.data.session.Definition object at 0x1772b8c40 {name='workspace'}>

In [22]:
horizon_date = None
variance_described = 0.0
residuals_values = pd.DataFrame()
residuals_z_scores = []

simulations = None


def getFPT(s0, Mean, Sigma, Lambda):
    sim = 150
    steps = 100
    mc = OUMonteCarlo(s0, Mean, Sigma, Lambda, sim, steps)
    temp_ftp = countwhile(steps, mc, Mean, sim)
    var = 0
    if len(temp_ftp) == 0:
        var = 0
    else:
        for i in range(sim):
            var += temp_ftp[i][0]
    return var / sim


def OUMonteCarlo(s0, Mean, Sigma, Lambda, sim, steps):
    global simulations
    # simulating an Ornstein-Uhlenbeck process
    mc = pd.DataFrame()
    for i in range(sim):
        _sim = []
        for j in range(steps):
            if j == 0:
                _sim.append(s0)
            else:
                W = norm.ppf(random.random())
                _sim.append(_sim[j-1] * np.exp(-Lambda) + Mean * (1 -
                            np.exp(-Lambda)) + np.sqrt(Sigma) * W)  # sqrt dt = 1
        mc[i] = _sim
    simulations = mc
    return mc


def countwhile(steps, mc, Mean, sim):
    arr = pd.DataFrame()
    for j in range(sim):
        i = 0
        while (abs(mc.iloc[i, j] - Mean) > 0) and (i < (steps - 1)):
            arr[j] = [i]
            i = i + 1
    return arr


def getMeanReversionParams(_df):
    Sx = Sy = Sxx = Syy = Sxy = 0
    _df = _df[0].tolist()
    for i in range(len(_df)):
        Sx = Sx + _df[i]
        Sxx = Sxx + _df[i] ** 2
        if i >= 1:
            Sxy = Sxy + _df[i] * _df[i-1]
    Sy = Sx - _df[0]
    Sx = Sx - _df[i]
    Syy = Sxx - _df[0] ** 2
    Sxx = Sxx - _df[i] ** 2
    n = i-1

    Mean = (Sy * Sxx - Sx * Sxy) / (n * (Sxx - Sxy) - (Sx ** 2 - Sx * Sy))
    Lambda = -np.log((Sxy - Mean * Sx - Mean * Sy + n * Mean **
                     2) / (Sxx - 2 * Mean * Sx + n * Mean ** 2))
    alpha = np.exp(-Lambda)
    sigma_tilda = 1 / n * (Syy - 2 * alpha * Sxy + Sxx * alpha ** 2 - 2 * Mean * (
        1 - alpha) * (Sy - alpha * Sx) + n * Mean ** 2 * (1 - alpha) ** 2)
    Sigma2 = sigma_tilda * 2 * Lambda / (1 - alpha ** 2)
    Sigma = np.sqrt(Sigma2)
    exp_hl = np.log(2) / Lambda
    exp_hl_90 = np.log(10) / Lambda

    return Mean, Lambda, Sigma2, exp_hl, exp_hl_90


def getMeanReversion(_df):
    global horizon_date
    rv_params = getMeanReversionParams(_df)
    Mean = rv_params[0]
    Lambda = rv_params[1]
    Sigma2 = rv_params[2]
    mrv = pd.DataFrame()
    date_array = []
    expected = []
    upper_2s = []
    lower_2s = []
    for t in range(252):
        _dt = datetime.timestamp(_df.index[-1])
        _dt = datetime.fromtimestamp(_dt)
        date_array.append(_dt + timedelta(days=t))
        if t == 0:
            expected.append(
                _df.iloc[-1][0] * np.exp(-Lambda) + Mean * (1 - np.exp(-Lambda)))
        else:
            expected.append(
                expected[t-1] * np.exp(-Lambda) + Mean * (1 - np.exp(-Lambda)))
        drift = 0.5 * Sigma2 / Lambda * (1 - np.exp(-2 * Lambda * t))
        upper_2s.append(expected[t] + 2 * np.sqrt(drift))
        lower_2s.append(expected[t] - 2 * np.sqrt(drift))

    holding_horizon = getFPT(_df.iloc[-1][0], Mean, Sigma2, Lambda)
    print('Optimal holding period (days): ' + str(round(holding_horizon, 1)))
    h_date = datetime.today() + timedelta(days=round(holding_horizon))
    print('Horizon date: ' + h_date.strftime('%Y-%m-%d'))
    horizon_date = h_date.strftime('%Y-%m-%d')
    mrv['Mean Reversion'] = expected
    mrv['+2 sigma'] = upper_2s
    mrv['-2 sigma'] = lower_2s
    mrv.index = date_array
    return mrv


def get_data(instruments, strategy, trade):
    global variance_described
    global residuals_values
    trade_dict = {'spread': 1, 'butterfly': 2}

    pc_limit = len(instruments)-1

    trade_dict = {'spread': 1, 'butterfly': 2}
    if pc_limit == 2:
        trade = 'butterfly'
    elif pc_limit == 1:
        trade = 'spread'

    _strategy = strategy
    strategy = strategy.split('-')
    strategy = list(map(lambda x: x+'Y', strategy))
    start = datetime.today() - timedelta(days=365)

    tnr = rd.get_data(instruments, 'GV4_TEXT')
    df = rd.get_history(instruments, 'B_YLD_1', start=start, count=10000)
    df = df.fillna(method='ffill')
    df = df.fillna(method='bfill')
    df_factors = df.sub(df.mean())
    df_diff = df.diff().dropna()

    pca = PCA(n_components=pc_limit, tol=1e-20, svd_solver='full')
    principal_components = pd.DataFrame(pca.fit_transform(df_diff))
    eigenvectors = pd.DataFrame(columns=instruments, data=pca.components_)

    df_pc = pd.DataFrame()
    for i in range(pc_limit):
        _pc = []
        for j in range(len(df_factors)):
            _pc.append(
                np.dot(eigenvectors.iloc[i, :].values, df_factors.iloc[j, :].values))
        df_pc[i] = _pc
    df_pc.index = df_factors.index

    principal_components = df_pc
    principal_components.index = df_factors.index
    principal_components = principal_components.T

    variance_described = pd.DataFrame(pca.explained_variance_ratio_).sum()
    eigenvalues = pd.DataFrame(pca.explained_variance_)

    residuals = pd.DataFrame(columns=instruments)
    for i, r in df_factors.iterrows():
        _v = []
        for c in eigenvectors.columns:
            _v.append(np.dot(eigenvectors[c], principal_components[i]))
        _res = pd.DataFrame(np.subtract(r.values, _v))
        _res.index = instruments
        _res.columns = [i]
        _res = _res.T
        residuals = pd.concat([residuals, _res])

    _tnr = list(map(str.strip, tnr['GV4_TEXT'].tolist()))
    residuals.columns = _tnr
    residuals_values = residuals * 100

    # residuals_z_scores =
    if len(strategy) == 3:
        df_str = pd.DataFrame(
            (residuals[strategy[0]] - (2 * residuals[strategy[1]]) + residuals[strategy[2]])*100)
    else:
        df_str = pd.DataFrame(
            (residuals[strategy[1]] - residuals[strategy[0]])*100)

    df_mrv = getMeanReversion(df_str)

    df_str = df_str.rename(columns={0: 'Strategy'})
    df_mrv['Strategy'] = np.nan
    df_str['Mean Reversion'] = np.nan
    df_str['+2 sigma'] = np.nan
    df_str['-2 sigma'] = np.nan

    res_vol = residuals_values.std()
    res_mu = residuals_values.mean()
    res_last = residuals_values.tail(1)
    residuals_z_scores = (res_last - res_mu) / res_vol
    df_out = pd.concat([df_str, df_mrv], sort=True)

    fig1 = df_out.iplot(vline=[horizon_date], yTitle='Residual, bps', title=_strategy+' '+trade, theme='solar',
                        colors=['#6978F7', '#A325E9', '#96E05D', '#4A7FB9', '#E75A2D', '#FBE55A', '#8C8C8D', '#5A54F6'])
    fig2 = residuals_values.tail(1).T.iplot(kind='bar', yTitle='Residual, bps', theme='solar', colors=[
        '#6978F7', '#A325E9', '#96E05D', '#4A7FB9', '#E75A2D', '#FBE55A', '#8C8C8D', '#5A54F6'])

    return df_out

In [ ]:
rd.close_session()